# Generate parquet datasets from original datasets

This notebook generates parquet datasets from original datasets, which ar stored in the data folder.

In [2]:
import pandas as pd
import dask.dataframe as dd

from dask.diagnostics import ProgressBar
ProgressBar().register()

KeyboardInterrupt: 

## Metadata

In [ ]:
METADATA_BLOCKSIZE = 2**27 # 128 MB

PATH_DATA = '../data/'

PATH_METADATA_SRC  = PATH_DATA + 'yt_metadata_en.jsonl'
PATH_METADATA_DST  = PATH_DATA + 'yt_metadata_en.parquet/'

### Write to file

In [ ]:
def cull_empty_partitions(df):
    ll = list(df.map_partitions(len).compute())
    df_delayed = df.to_delayed()
    df_delayed_new = list()
    pempty = None
    for ix, n in enumerate(ll):
        if 0 == n:
            pempty = df.get_partition(ix)
        else:
            df_delayed_new.append(df_delayed[ix])
    if pempty is not None:
        df = dd.from_delayed(df_delayed_new, meta=pempty)
    return df

In [ ]:
dtype = {
    'categories'    : 'str',
    'channel_id'    : 'str',
    'crawl_date'    : 'str',
    'description'   : 'str',
    'dislike_count' : 'int32',
    'display_id'    : 'str',
    'duration'      : 'int32',
    'like_count'    : 'int32',
    'tags'          : 'str',
    'title'         : 'str',
    'upload_date'   : 'str',
    'view_count'    : 'int32'
}
df = dd.read_json(PATH_METADATA_SRC, lines=True, blocksize=METADATA_BLOCKSIZE, dtype=dtype, convert_dates=['crawl_date', 'upload_date'])
df['categories'] = df['categories'].astype('category')
df = cull_empty_partitions(df)

In [ ]:
df.to_parquet(PATH_METADATA_DST, engine='pyarrow', compression='snappy')

### Read from file